In [ ]:
print('Starting...')

from dataclasses import dataclass
from typing import List, Dict, Optional, Tuple, Literal
from Pynite import FEModel3D
from Pynite.Rendering import Renderer
import numpy as np
import pandas as pd
from collections import defaultdict
from skopt import gp_minimize
from skopt.space import Integer, Categorical
from skopt.utils import use_named_args
import warnings
import bisect
import math
from tqdm import tqdm

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import HuberRegressor


@dataclass
class DesignParameters:
    room_length: float
    room_width: float
    room_height: float
    plank_thickness: float
    opening_width: float
    opening_length: float
    opening_x_start: float
    wall_beam_contact_depth: float
    live_load_mpa: float
    wall_thickness: float

    @property
    def floor_z(self):
        return self.room_height / 2

    @property
    def beam_length(self):
        return self.room_width + self.wall_beam_contact_depth
    
    @property
    def opening_y(self):
        return self.beam_length - self.opening_width - self.wall_beam_contact_depth/2


def prep_data():
    # Units are mm, N, and MPa (N/mm²)
    # INPUT PARAMS
    input_params = pd.read_csv('data/design_parameters.csv').iloc[0].to_dict()
    input_params = DesignParameters(**input_params)

    # MATERIAL STRENGTH
    material_str = pd.read_csv('data/material_strengths.csv')
    material_str['G_05'] = material_str['G'] * 5/6
    material_str['E_05'] = material_str['E_0'] * 2/3
    material_str = material_str.set_index('material').to_dict(orient='index')

    # CONNECTORS
    connectors = pd.read_csv('data/connectors.csv')
    train_data = connectors.dropna(subset=['grams'])
    predict_data = connectors[connectors['grams'].isnull()].copy()

    ## Fill missing weights
    X_train = train_data[['base', 'height']]
    y_train = train_data['grams']
    X_predict = predict_data[['base', 'height']]
    model = LinearRegression()
    model.fit(X_train, y_train)
    predict_data['grams'] = model.predict(X_predict)
    connectors = pd.concat([train_data, predict_data])

    ## Smoothen price and groupby base and height
    X = connectors[['base', 'height']]
    y = connectors['price_unit']
    huber = HuberRegressor(epsilon=1.24)
    huber.fit(X, y)
    connectors['price_unit'] = huber.predict(X)
    connectors['weight_N'] = connectors['grams'] / 100
    connectors = connectors[['base', 'height', 'weight_N', 'price_unit']].groupby(['base', 'height']).mean()
    connectors = connectors.reset_index()

    # MATERIAL_CATALOG
    material_catalog = pd.read_csv('data/material_catalog.csv')
    material_catalog['id'] = (material_catalog['material'] + '_' + material_catalog['base'].astype(str) + 'x' + material_catalog['height'].astype(str))

    ## Create fitting double beams
    doubled_beams = material_catalog[(material_catalog['type'] == 'beam') & (material_catalog['material'] != 'steel')].copy()
    doubled_beams['base'] = doubled_beams['base'] * 2
    doubled_beams['type'] = 'double'
    doubled_beams['source'] = doubled_beams['id']
    material_catalog = pd.concat([material_catalog, doubled_beams], ignore_index=True)
    material_catalog['viable_connector'] = material_catalog['base'].isin(connectors['base'])

    ## Standardize floor materials to 200mm width
    is_floor = material_catalog['type'] == 'floor'
    material_catalog.loc[is_floor, 'cost_unit'] = material_catalog.loc[is_floor, 'cost_unit'] * (200 / material_catalog.loc[is_floor, 'base'])
    material_catalog.loc[is_floor, 'base'] = 200

    material_catalog['id'] = (material_catalog['material'] + '_' + material_catalog['base'].astype(str) + 'x' + material_catalog['height'].astype(str))

    # EUROCODE FACTORS
    eurocode_factors = pd.read_csv('data/eurocode_material_factors.csv').set_index('material_prefix').to_dict(orient='index')

    return input_params, material_str, material_catalog, connectors, eurocode_factors


def _get_eurocode_factors(material_name):
    if material_name.startswith('c'):
        return EUROCODE_FACTORS['c']
    elif material_name.startswith('gl'):
        return EUROCODE_FACTORS['gl']
    elif material_name.startswith('osb'):
        return EUROCODE_FACTORS['osb']
    elif material_name.startswith('p'):
        return EUROCODE_FACTORS['p']
    elif material_name.startswith('s'):
        return EUROCODE_FACTORS['s']
    elif material_name.startswith('brick'):
        return EUROCODE_FACTORS['brick']
    else:
        raise ValueError("Material not supported")

@dataclass
class CrossSectionProperties:
    A: float
    Iy: float
    Iz: float
    J: float
    
    @classmethod
    def from_rectangular(cls, base: float, height: float) -> 'CrossSectionProperties':
        A = base * height
        b, h = min(base, height), max(base, height)
        J = (b**3 * h) * (1/3 - 0.21 * (b/h) * (1 - (b**4)/(12*h**4)))
        Iz = (height * base**3) / 12
        Iy = (base * height**3) / 12
        return cls(A=A, Iy=Iy, Iz=Iz, J=J)
    
    # @classmethod
    # def from_i_beam(cls, height: float, base: float, flange_thickness: float, web_thickness: float) -> 'CrossSectionProperties':
    #     A_flanges = 2 * base * flange_thickness
    #     A_web = (height - 2 * flange_thickness) * web_thickness
    #     A = A_flanges + A_web
        
    #     Iz_flanges = 2 * (base * flange_thickness**3 / 12 + 
    #                      base * flange_thickness * ((height - flange_thickness)/2)**2)
    #     web_height = height - 2 * flange_thickness
    #     Iz_web = web_thickness * web_height**3 / 12
    #     Iz = Iz_flanges + Iz_web
        
    #     Iy_flanges = 2 * (flange_thickness * base**3 / 12)
    #     Iy_web = web_height * web_thickness**3 / 12
    #     Iy = Iy_flanges + Iy_web
        
    #     J = (2 * base * flange_thickness**3 + web_height * web_thickness**3) / 3
    #     return cls(A=A, Iy=Iy, Iz=Iz, J=J)

@dataclass
class MemberSpec:
    material_id: str
    quantity: Optional[int] = None
    padding: Optional[float] = None
    
    def __post_init__(self):
        self._catalog_data = MATERIAL_CATALOG[MATERIAL_CATALOG['id'] == self.material_id].iloc[0]
        self.material = self._catalog_data['material']
        self.type = self._catalog_data['type']
        self.base = self._catalog_data['base']
        self.height = self._catalog_data['height']
        self.length = self._catalog_data['length']
        self.shape = self._catalog_data['shape']
        self.cost_per_m3 = self._catalog_data['cost_unit']
    
    @property
    def section_name(self) -> str:
        return f"sec_{self.material_id}"
    
    def get_geometry(self) -> CrossSectionProperties:
        if self.shape == 'rectangular':
            return CrossSectionProperties.from_rectangular(self.base, self.height)
        # elif self.shape.startswith('IP'):
        #     return CrossSectionProperties.from_i_beam(self.height, self.base, self._catalog_data['flange_thickness'], self._catalog_data['web_thickness'])
        else:
            raise ValueError(f"Unknown shape: {self.shape}")
    
    def create_section(self, frame: FEModel3D):
        if self.section_name not in frame.sections:
            geom = self.get_geometry()
            frame.add_section(self.section_name, geom.A, Iy=geom.Iy, Iz=geom.Iz, J=geom.J)

@dataclass
class NodeLocation:
    name: str
    X: float
    Y: float
    Z: float

@dataclass
class Member:
    name: str
    node_i: str
    node_j: str
    spec: MemberSpec
    

def calculate_nodes_and_members(hyperparams: dict) -> Tuple[List[NodeLocation], List[Member]]:

    def _calculate_evenly_spaced_positions(
        n: int, 
        clear_start: float, 
        clear_end: float, 
        member_base: float,
        distribution: Literal['start_aligned', 'end_aligned', 'centered'],
        tail_padding=None) -> List[float]:

        if n == 0:
            return []
        
        if distribution == 'start_aligned':
            centerline_start = clear_start + member_base / 2
            centerline_end = clear_end + member_base / 2
            positions = np.linspace(centerline_start, centerline_end, n + 1).tolist()
            return positions[:-1]
        
        elif distribution == 'end_aligned':
            centerline_start = clear_start - member_base / 2
            centerline_end = clear_end - member_base / 2
            positions = np.linspace(centerline_start, centerline_end, n + 1).tolist()
            return positions[1:]
        
        elif distribution == 'centered':
            centerline_start = clear_start + member_base / 2
            centerline_end = clear_end - member_base / 2
            if tail_padding:
                positions = np.linspace(centerline_start, centerline_end, n).tolist()
                return positions
            positions = np.linspace(centerline_start, centerline_end, n + 2).tolist()
            return positions[1:-1]
        else:
            raise ValueError(f"Unknown distribution: {distribution}")

    assert hyperparams['trimmers'].quantity == 2, "Must have exactly 2 trimmers"
    assert hyperparams['header'].quantity == 1, "Must have exactly 1 header"
    
    # Stairs opening params
    opening_x_end = INPUT_PARAMS.opening_x_start + INPUT_PARAMS.opening_length
    trimmer_W_x = INPUT_PARAMS.opening_x_start - (hyperparams['trimmers'].base / 2)
    trimmer_E_x = opening_x_end + (hyperparams['trimmers'].base / 2)

    header_y = INPUT_PARAMS.opening_y - hyperparams['header'].base / 2
    
    # Beam centerline positions
    beam_positions = {}
    if hyperparams['west_joists'].quantity > 0:
        clear_start = hyperparams['west_joists'].padding
        clear_end = trimmer_W_x - hyperparams['trimmers'].base / 2
        x_positions = _calculate_evenly_spaced_positions(hyperparams['west_joists'].quantity,
                                                         clear_start,
                                                         clear_end,
                                                         hyperparams['west_joists'].base,
                                                         'start_aligned')
        beam_positions['west_joists'] = [(f'west{i}', x) for i, x in enumerate(x_positions)]
    
    if hyperparams['tail_joists'].quantity > 0:
        clear_start = INPUT_PARAMS.opening_x_start + hyperparams['tail_joists'].padding
        clear_end = opening_x_end - hyperparams['tail_joists'].padding
        x_positions = _calculate_evenly_spaced_positions(hyperparams['tail_joists'].quantity,
                                                         clear_start,
                                                         clear_end,
                                                         hyperparams['tail_joists'].base,
                                                         'centered',
                                                         tail_padding=hyperparams['tail_joists'].padding)
        beam_positions['tail_joists'] = [(f'tail{i}', x) for i, x in enumerate(x_positions)]
    
    if hyperparams['east_joists'].quantity > 0:
        clear_start = trimmer_E_x + hyperparams['trimmers'].base / 2
        clear_end = INPUT_PARAMS.room_length - hyperparams['east_joists'].padding
        x_positions = _calculate_evenly_spaced_positions(hyperparams['east_joists'].quantity,
                                                         clear_start,
                                                         clear_end,
                                                         hyperparams['east_joists'].base,
                                                         'end_aligned')
        beam_positions['east_joists'] = [(f'east{i}', x) for i, x in enumerate(x_positions)]
    
    beam_positions['trimmers'] = [('trimmerE', trimmer_E_x), ('trimmerW', trimmer_W_x)]

    # Plank centerline positions
    clear_start = INPUT_PARAMS.wall_beam_contact_depth / 2
    clear_end = INPUT_PARAMS.beam_length - INPUT_PARAMS.wall_beam_contact_depth / 2
    hyperparams['planks'].quantity = int((clear_end - clear_start) // hyperparams['planks'].base)
    y_positions = _calculate_evenly_spaced_positions(hyperparams['planks'].quantity,
                                                     clear_start-hyperparams['planks'].base,
                                                     clear_end+hyperparams['planks'].base,
                                                     hyperparams['planks'].base,
                                                     'centered')
    plank_positions = [(f'p{i}', y) for i, y in enumerate(y_positions)]
    
    # Beam nodes and member locations
    nodes = []
    members = []
    for group_name, group_positions in beam_positions.items():
        spec = hyperparams[group_name]
        for beam_name, x in group_positions:
            nodes.append(NodeLocation(f'{beam_name}_S', x, 0, INPUT_PARAMS.floor_z))
            if group_name == 'tail_joists': # Tails connect to header (header_y), not to wall (beam_length)
                nodes.append(NodeLocation(f'{beam_name}_header', x, header_y, INPUT_PARAMS.floor_z))
                members.append(Member(name=beam_name, node_i=f'{beam_name}_S', node_j=f'{beam_name}_header', spec=spec))
            else:
                nodes.append(NodeLocation(f'{beam_name}_N', x, INPUT_PARAMS.beam_length, INPUT_PARAMS.floor_z))
                members.append(Member(name=beam_name, node_i=f'{beam_name}_S', node_j=f'{beam_name}_N', spec=spec))
    
    nodes.append(NodeLocation('headerE', trimmer_E_x, header_y, INPUT_PARAMS.floor_z))
    nodes.append(NodeLocation('headerW', trimmer_W_x, header_y, INPUT_PARAMS.floor_z))
    members.append(Member(name='header', node_i='headerW', node_j='headerE', spec=hyperparams['header']))

    # Corner nodes
    walls = [('W', 0), ('E', INPUT_PARAMS.room_length)]
    for corner_name, x in walls:
        nodes.append(NodeLocation(f'{corner_name}_S', x, 0, INPUT_PARAMS.floor_z))
        nodes.append(NodeLocation(f'{corner_name}_N', x, INPUT_PARAMS.beam_length, INPUT_PARAMS.floor_z))

    # Plank nodes and member locations
    plank_series_dict = {}
    beam_positions['walls'] = walls # So planks extend to walls
    for plank_name, y in plank_positions:
        plank_nodes = []
        for group_positions in beam_positions.values():
            for beam_name, x in group_positions:
                if x > trimmer_W_x and x < trimmer_E_x and y > header_y:
                    continue
                intersection_node = NodeLocation(f'{beam_name}-{plank_name}', x, y, INPUT_PARAMS.floor_z)
                plank_nodes.append(intersection_node)
                nodes.append(intersection_node)
        plank_series_dict[plank_name] = plank_nodes
    
    for plank_name, plank_nodes in plank_series_dict.items():
        plank_nodes.sort(key=lambda node: node.X)
        for i, _ in enumerate(plank_nodes[:-1]):
            if plank_nodes[i].X == trimmer_W_x and plank_nodes[i].Y > header_y:
                continue
            members.append(Member(name=f'{plank_name}_{i}', node_i=plank_nodes[i].name, node_j=plank_nodes[i+1].name, spec=hyperparams['planks']))
    
    return nodes, members

    
def assemble_frame(nodes: List[NodeLocation], members: List[Member]) -> Tuple[FEModel3D, Dict[str, MemberSpec]]:
    frame = FEModel3D()
    for mat_name, props in MATERIAL_STRENGTHS.items():
        if props['G'] is None:
            G = props['E_0'] / (2 * (1 + props['nu']))
        else:
            G = props['G']
        frame.add_material(
            mat_name,
            E=props['E_0'],
            G=G,
            nu=props['nu'],
            rho=props['rho']
        )
    
    for node in nodes:
        frame.add_node(node.name, node.X, node.Y, node.Z)
    
    for member in members:
        member.spec.create_section(frame)
        frame.add_member(
            member.name,
            member.node_i,
            member.node_j,
            member.spec.material,
            member.spec.section_name
        )
    
    return frame


def define_supports(frame, nodes, wall_thickness, material, walls=False) -> None:
    supports = {}
    supports['north'] = sorted([n for n in nodes if n.name.endswith('_N')], key=lambda n: n.X)
    supports['south'] = sorted([n for n in nodes if n.name.endswith('_S')], key=lambda n: n.X)
    supports['east'] = sorted([n for n in nodes if n.name.startswith('E_')], key=lambda n: n.Y)
    supports['west'] = sorted([n for n in nodes if n.name.startswith('W_')], key=lambda n: n.Y)

    if walls:
        foundation = []
        for support_side, support_nodes in supports.items():
            for node in support_nodes:
                if node.name not in foundation:
                    foundation.append(node.name)
                    frame.add_node(f'{node.name}_foundation', node.X, node.Y, 0)

        for support_side, support_nodes in supports.items():
            for i, _ in enumerate(support_nodes[:-1]):
                frame.add_quad(
                    f'{support_side}_wall{i}',
                    support_nodes[i].name,
                    f'{support_nodes[i].name}_foundation',
                    f'{support_nodes[i+1].name}_foundation',
                    support_nodes[i+1].name,
                    wall_thickness, 
                    material
                )
        for node_name in frame.nodes:
            if node_name.endswith('_foundation'):
                frame.def_support(node_name, True, True, True, True, True, True)
    else:
        for support_side, support_nodes in supports.items():
            if support_side in ['north', 'south']:
                for node in support_nodes:
                    frame.def_support(node.name, True, True, True, True, True, True)


def _find_compatible_connector(base: float, height: float):
    member_connectors = CONNECTORS[CONNECTORS['base'] == base]
    connector = member_connectors[member_connectors['height'] <= height]
    if connector.empty:
        return member_connectors.mean()
    return connector.mean()
    

def apply_loads(frame: FEModel3D, members: List[Member]) -> Tuple[float, float]:
    total_dl_force, total_ll_force = 0.0, 0.0

    # Dead loads
    for member in members:
        geom = member.spec.get_geometry()
        material = MATERIAL_STRENGTHS[member.spec.material]
        dead_load = -geom.A * material['rho']
        frame.add_member_dist_load(member.name, 'FZ', dead_load, dead_load, case=DL_COMBO)
        total_dl_force += dead_load * frame.members[member.name].L()

        if member.name.startswith('tail'):
            header = next((m for m in members if m.name.startswith('header')))
            connector = _find_compatible_connector(base=member.spec.base, height=header.spec.height)
            frame.add_member_pt_load(member.name, 'FZ', -connector['weight_N'], frame.members[member.name].L(), case=DL_COMBO)
            total_dl_force += -connector['weight_N']
        elif member.name.startswith('header'):
            trimmer = next((m for m in members if m.name.startswith('trimmer')))
            connector = _find_compatible_connector(base=member.spec.base, height=trimmer.spec.height)
            frame.add_member_pt_load(member.name, 'FZ', -connector['weight_N'], 0, case=DL_COMBO)
            frame.add_member_pt_load(member.name, 'FZ', -connector['weight_N'], frame.members[member.name].L(), case=DL_COMBO)
            total_dl_force += -connector['weight_N'] * 2

    # Live loads
    plank_members = [m for m in members if m.name.startswith('p')]
    plank_y_values = sorted(set(frame.nodes[m.node_i].Y for m in plank_members))
    standard_spacing = plank_y_values[1] - plank_y_values[0]
    
    tail_planks = []
    for m in plank_members:
        if 'tail' in m.node_i or 'tail' in m.node_j or ('trimmer' in m.node_i and 'trimmer' in m.node_j):
            tail_planks.append(m)
    
    min_y = min(plank_y_values)
    max_y = max(plank_y_values)
    max_tail_plank_y = max(frame.nodes[m.node_i].Y for m in tail_planks)
    for member in plank_members:
        member_y = frame.nodes[member.node_i].Y
        
        if member_y == min_y:
            tributary_width = member.spec.base / 2 + standard_spacing / 2
        elif member_y == max_y:
            tributary_width = member.spec.base / 2 + standard_spacing / 2
        elif member_y == max_tail_plank_y and member in tail_planks:
            tributary_width = INPUT_PARAMS.opening_y - member_y + standard_spacing / 2
        else:
            tributary_width = standard_spacing
        
        live_load = -INPUT_PARAMS.live_load_mpa * tributary_width
        frame.add_member_dist_load(member.name, 'FZ', live_load, live_load, case=LL_COMBO)
        total_ll_force += live_load * frame.members[member.name].L()

    # Declare loads
    frame.add_load_combo(LL_COMBO, {'LL': 1})
    frame.add_load_combo(DL_COMBO, {'DL': 1})
    frame.add_load_combo(ULS_COMBO, {'DL': 1.35, 'LL': 1.5})

    return total_dl_force, total_ll_force


def create_model(hyperparams: dict, walls: bool = True) -> Tuple:
    
    nodes, members = calculate_nodes_and_members(hyperparams)
    frame = assemble_frame(nodes, members)
    define_supports(frame, nodes, INPUT_PARAMS.wall_thickness, 'brick', walls=walls)
    total_dl_force, total_ll_force = apply_loads(frame, members)
    
    return frame, nodes, members


def calculate_purchase_quantity(frame: FEModel3D, members: List[Member]):

    def _find_optimal_cuts(member_lengths: List[float], stock_length: float):
        member_lengths.sort(reverse=True)
        stock_parts = []
        remaining_lengths_in_stock_parts = []
        for member_len in member_lengths:
            index_for_best_stock_part = bisect.bisect_left(remaining_lengths_in_stock_parts, member_len)
            
            if index_for_best_stock_part < len(remaining_lengths_in_stock_parts):
                old_remaining_lengths = remaining_lengths_in_stock_parts.pop(index_for_best_stock_part)
                best_stock_part = stock_parts.pop(index_for_best_stock_part)
                best_stock_part.append(member_len)

                new_remaining_length = old_remaining_lengths - member_len
                
                new_index_for_stock_part = bisect.bisect_left(remaining_lengths_in_stock_parts, new_remaining_length)
                remaining_lengths_in_stock_parts.insert(new_index_for_stock_part, new_remaining_length)
                stock_parts.insert(new_index_for_stock_part, best_stock_part)
                
            else:
                new_remaining_length = stock_length - member_len
                
                new_insertion_index = bisect.bisect_left(remaining_lengths_in_stock_parts, new_remaining_length)
                remaining_lengths_in_stock_parts.insert(new_insertion_index, new_remaining_length)
                stock_parts.insert(new_insertion_index, [member_len])

        return len(stock_parts), stock_parts
    
    total_cost = 0.0
    materials = defaultdict(list)
    for member in members:
        pynite_member = frame.members[member.name]
        member_length = float(pynite_member.L())

        if member.spec.type == 'double':
            source_id = MATERIAL_CATALOG[MATERIAL_CATALOG['id'] == member.spec.material_id].iloc[0]['source']
            materials[source_id].extend([member_length] * 2)
        else:
            material_id = member.spec.material_id
            materials[material_id].extend([member_length])

        if member.name.startswith('tail'):
            header = next((m for m in members if m.name.startswith('header')))
            connector = _find_compatible_connector(base=member.spec.base, height=header.spec.height)
            total_cost += connector['price_unit']
        elif member.name.startswith('header'):
            trimmer = next((m for m in members if m.name.startswith('trimmer')))
            connector = _find_compatible_connector(base=member.spec.base, height=trimmer.spec.height)
            total_cost += connector['price_unit'] * 2

    all_material_cuts = {}
    for material_id, lengths in materials.items():
        material_specs = MATERIAL_CATALOG[MATERIAL_CATALOG['id'] == material_id].iloc[0]
        num_beams_to_buy, current_material_cuts = _find_optimal_cuts(lengths, material_specs['length'])
        all_material_cuts[material_id] = current_material_cuts
        total_cost += num_beams_to_buy * material_specs['cost_unit']
    
    return total_cost, all_material_cuts


def evaluate_stresses(frame: FEModel3D, members: List[Member]):
    # Following chapters in https://www.swedishwood.com/siteassets/5-publikationer/pdfer/sw-design-of-timber-structures-vol2-2022.pdf

    def _calc_size_factor_kh(spec):
        """
        Chapter 3.3 Size effects (EN 1995-1-1, 3.2-3.4)
        Used for bending and tension

        k_h : size factor 
        """
        if spec.material.startswith('c') and spec.height <= 150:
            k_h = min((150/spec.height)**0.2, 1.3)
        elif spec.material.startswith('gl') and spec.height <= 600:
            k_h = min((600/spec.height)**0.1, 1.1)
        else:
            k_h = 1
        return k_h


    def _calc_bending_moment_capacity(factors, material_props, spec, geometry, member_length):
        """
        Chapter 4 Bending (EN 1995-1-1, 6.3.3)
        
        W_y : section modulus about the major axis (y)
        W_z : section modulus about the minor axis (z)
        M_y_crit : critical bending moment about the major axis (y)
        l_ef : effective length of the beam for a uniformly distributed load
        omega_m_crit : critical bending stress calculated according to the classical theory of lateral stability, using 5-percentile stiffness values (EN 1995-1-1, 6.3.3)
        lambda_rel_m : relative slenderness ratio in bending
        k_crit : factor accounting for the effect of lateral buckling
        f_mk : characteristic bending strength
        f_md : design value of bending strength

        M_yRd : bending moment capacity about the major axis (y)
        M_zRd : bending moment capacity about the minor axis (z)
        """
        W_y = geometry.Iy / (spec.height / 2)
        W_z = geometry.Iz / (spec.base / 2)
        M_y_crit = math.pi * math.sqrt(material_props['E_05'] * geometry.Iz * material_props['G_05'] * geometry.J)

        l_ef = member_length * 0.9
        omega_m_crit = M_y_crit / (W_y * l_ef)

        lambda_rel_m = math.sqrt(material_props['f_mk'] / omega_m_crit)
        if lambda_rel_m <= 0.75:
            k_crit = 1.0
        elif lambda_rel_m <= 1.4:
            k_crit = 1.56 - 0.75 * lambda_rel_m
        else:
            k_crit = 1 / lambda_rel_m ** 2

        k_h = _calc_size_factor_kh(spec)
        k_mod = factors['k_mod']
        gamma_M = factors['gamma_M']
        f_md = (k_h * material_props['f_mk'] * k_mod) / gamma_M
        M_yRd = f_md * W_y * k_crit
        M_zRd = f_md * W_z

        return M_yRd, M_zRd


    def _calc_axial_tension_capacity(factors, material_props, spec, geometry):
        """
        Chapter 5.1 Axial Tension

        f_t0k : characteristic tension strength parallel to the grain
        f_t0d : design tension strength parallel to the grain
        f_t90k : characteristic tension strength perpendicular to the grain
        f_t90d : design tension strength perpendicular to the grain

        N_t0Rd : design load capacity in axial tension about the strong axis (y)
        N_t90Rd : design load capacity in axial tension about the weak axis (z)
        """
        k_mod = factors['k_mod']
        gamma_M = factors['gamma_M']
        k_h = _calc_size_factor_kh(spec)

        f_t0d = (k_h * material_props['f_t0k'] * k_mod) / gamma_M
        f_t90d = (k_h * material_props['f_t90k'] * k_mod) / gamma_M
        N_t0Rd = f_t0d * geometry.A
        N_t90Rd = f_t90d * geometry.A

        return N_t0Rd, N_t90Rd


    def _calc_instability_factor(factors, material_props, geometry, member_length):
        """
        Chapter 5.2 Compression (EN 1995-1-1, 6.3.2)

        i_y : radius of gyration about the strong axis (y)
        i_z : radius of gyration about the weak axis (z)
        l_ef : effective length
        lambda_y, lambda_rel_y : slenderness ratios corresponding to bending about the y-axis (deflection in the z-direction)
        lambda_z, lambda_rel_z : slenderness ratios corresponding to bending about the z-axis (deflection in the y-direction)
        f_c0k : characteristic compression strength parallel to grain

        k, k_c : instability factors
        """
        beta_c = factors['straightness_factor_beta_c']
        
        i_y = math.sqrt(geometry.Iy / geometry.A)
        i_z = math.sqrt(geometry.Iz / geometry.A)
        l_ef = member_length * 0.9
        lambda_y = l_ef / i_y
        lambda_z = l_ef / i_z
        lambda_y_rel = (lambda_y / math.pi) * math.sqrt(material_props['f_c0k'] / material_props['E_05'])
        lambda_z_rel = (lambda_z / math.pi) * math.sqrt(material_props['f_c0k'] / material_props['E_05'])
        risk_buckling = lambda_y_rel > 0.3 or lambda_z_rel > 0.3

        k_y = (0.5 * (1 + beta_c * (lambda_y_rel - 0.3) + lambda_y_rel**2))
        k_z = (0.5 * (1 + beta_c * (lambda_z_rel - 0.3) + lambda_z_rel**2))
        k_cy = 1 / (k_y + math.sqrt(k_y**2 - lambda_y_rel**2))
        k_cz = 1 / (k_z + math.sqrt(k_z**2 - lambda_z_rel**2))

        return k_cy, k_cz, risk_buckling


    def _calc_axial_compression_capacity(factors, material_props, spec, geometry, member_length):
        """
        Chapter 5.2 Axial Compression (EN 1995-1-1, 6.1.5)
        
        f_c0k : characteristic compression strength parallel to grain
        f_c0d : design compression strength parallel to grain
        f_c90k : characteristic compression strength perpendicular to grain
        f_c90d : design compression strength perpendicular to grain
        k_cz : instability factor c about the weak axis (z)
        k_cy : instability factor c about the strong axis (y)
        A_ef : effective contact area in compression

        N_c0Rd : design load capacity in axial compression about the strong axis (y)
        N_c90Rd : design load capacity in axial compression about the weak axis (z)
        """
        k_mod = factors['k_mod']
        gamma_M = factors['gamma_M']
        f_c0d = (material_props['f_c0k'] * k_mod) / gamma_M
        f_c90d = (material_props['f_c90k'] * k_mod) / gamma_M
        k_c90 = factors['config_and_deformation_factor_k_c90']
        k_cy, k_cz, risk_buckling = _calc_instability_factor(factors, material_props, geometry, member_length)

        N_c0Rd = f_c0d * geometry.A * k_cy
        A_ef = spec.base * INPUT_PARAMS.wall_beam_contact_depth
        N_c90Rd = k_c90 * f_c90d * A_ef

        return N_c0Rd, N_c90Rd, k_cy, k_cz, risk_buckling


    def _calc_shear_ratio(factors, material_props, spec, V_Ed):
        """
        Chapter 6 Cross section subjected to shear (EN 1995-1-1, 6.1.7)
        
        b_ef : effective width for area calculations
        f_vk : characteristic shear strength
        f_vd : design shear strength
        V_Rd : design load capacity in shear strength
        """
        k_mod = factors['k_mod']
        gamma_M = factors['gamma_M']
        k_cr = factors['k_cr']

        if spec.material.startswith('c') or spec.material.startswith('gl'):
            b_ef = spec.base * k_cr
        else:
            b_ef = spec.base

        f_vd = (material_props['f_vk'] * k_mod) / gamma_M
        V_Rd = f_vd * (b_ef * spec.height) / 1.5

        return V_Ed / V_Rd
    

    def _calc_torsion_ratio(factors, material_props, spec, T_Ed):
        """
        (EN 1995-1-1, 6.2.4, 6.1.8)

        k_shape : factor depending on the shape of the cross-section
        W_t : torsional section modulus of a solid rectangular section
        f_vk : characteristic shear strength
        tau_tor_d : design torsional stress
        T_Rd : design load capacity in torsional strength
        """
        k_shape = min(1 + (0.15 * spec.height/spec.base), 2)
        k_t = 1 / (3 + 1.8 * (spec.base / spec.height))
        W_t = spec.height * spec.base**2 * k_t
        f_vd = (material_props['f_vk'] * factors['k_mod']) / factors['gamma_M']
        tau_tor_d = (T_Ed / W_t)
        T_Rd = (k_shape * f_vd)

        return tau_tor_d / T_Rd


    def _calc_combined_bending_axial_tension_ratio(M_yRd, M_zRd, N_t0Rd, M_yEd, M_zEd, N_t0Ed, k_m):
        """
        Chapter 7.2 Combined bending and axial tension (EN 1995-1-1, 6.2.3)

        k_m: reduction factor depending on cross-sectional shape

        M_yEd : design load effect from bending moments about the strong axis (y)
        M_zEd : design load effect from bending moments about the weak axis (z)
        N_t0Ed : design load effect from axial tension about the strong axis (y)

        M_yRd : design load capacity in bending moments about the strong axis (y)
        M_zRd : design load capacity in bending moments about the weak axis (z)
        N_t0Rd : design load capacity in axial tension about the strong axis (y)

        ratio_y, ratio_z : ratios for member stress over member capacity
        Member failure occurs when ratio_y or ratio_z exceed a value of 1
        """
        ratio_y = (M_yEd * k_m / M_yRd) + (M_zEd / M_zRd) + (N_t0Ed / N_t0Rd)
        ratio_z = (M_yEd / M_yRd) + (M_zEd * k_m / M_zRd) + (N_t0Ed / N_t0Rd)

        return ratio_y, ratio_z


    def _calc_combined_bending_axial_compression_ratio(risk_buckling, M_yRd, M_zRd, N_c0Rd, M_yEd, M_zEd, N_c0Ed, k_cy, k_cz, k_m):
        """
        Chapter 7.3 Combined bending and axial compression (EN 1995-1-1, 6.2.4, 6.3.2)

        k_m : reduction factor depending on cross-sectional shape
        k_cz : instability factor c about the weak axis (z)
        k_cy : instability factor c about the strong axis (y)

        M_yEd : design load effect from bending moments about the strong axis (y)
        M_zEd : design load effect from bending moments about the weak axis (z)
        N_c0Ed : design load effect from axial compression about the strong axis (y)

        M_yRd : design load capacity in bending moments about the strong axis (y)
        M_zRd : design load capacity in bending moments about the weak axis (z)
        N_c0Rd : design load capacity in axial compression about the strong axis (y)

        ratio_y, ratio_z : ratios for member stress over member capacity
        Member failure occurs when ratio_y or ratio_z exceed a value of 1
        """
        if risk_buckling:
            ratio_y = (M_yEd * k_m / M_yRd) + (M_zEd / M_zRd) + (N_c0Ed / (N_c0Rd * k_cz))
            ratio_z = (M_yEd / M_yRd) + (M_zEd * k_m / M_zRd) + (N_c0Ed / (N_c0Rd * k_cy))
        else:
            ratio_y = (M_yEd * k_m / M_yRd) + (M_zEd / M_zRd) + (N_c0Ed / N_c0Rd)**2
            ratio_z = (M_yEd / M_yRd) + (M_zEd * k_m / M_zRd) + (N_c0Ed / N_c0Rd)**2

        return ratio_y, ratio_z


    def _calc_compression_bearing_ratio(factors, material_props, spec, pynite_member, member_end):
        brick_props = MATERIAL_STRENGTHS['brick']
        mortar_props = MATERIAL_STRENGTHS['mortar']
        brick_gamma_M = 2.2
        alpha = 0.7
        beta = 0.3
        K = 0.45

        bearing_area = spec.base * INPUT_PARAMS.wall_beam_contact_depth    
        reaction_force_j = abs(pynite_member.shear('Fz', member_end, ULS_COMBO))
        bearing_pressure_j = reaction_force_j / bearing_area

        f_c90d_beam = (material_props['f_c90k'] * factors['k_mod']) / factors['gamma_M']
        f_kd = K * brick_props['f_c0k']**alpha * mortar_props['f_c0k']**beta / brick_gamma_M

        beam_bearing_ratio = bearing_pressure_j / f_c90d_beam
        brick_bearing_ratio = bearing_pressure_j / f_kd

        return beam_bearing_ratio, brick_bearing_ratio


    frame.analyze(check_stability=False)
    support_node_names = {node.name for node in frame.nodes.values() if node.name.endswith(('_N', '_S'))}
    
    member_evaluations = []
    for member in members:
        ratios = {}
        pynite_member = frame.members[member.name]
        spec = member.spec
        geometry = spec.get_geometry()
        material_props = MATERIAL_STRENGTHS[spec.material]
        factors = _get_eurocode_factors(spec.material)
        member_length = pynite_member.L()

        # ULS Internal Forces
        max_moment_y = pynite_member.max_moment('My', ULS_COMBO)
        min_moment_y = pynite_member.min_moment('My', ULS_COMBO)
        M_yEd = max(abs(max_moment_y), abs(min_moment_y))

        max_moment_z = pynite_member.max_moment('Mz', ULS_COMBO)
        min_moment_z = pynite_member.min_moment('Mz', ULS_COMBO)
        M_zEd = max(abs(max_moment_z), abs(min_moment_z))

        N_t0Ed = abs(min(0, pynite_member.min_axial(ULS_COMBO)))
        N_c0Ed = max(0, pynite_member.max_axial(ULS_COMBO))

        max_shear_z = pynite_member.max_shear('Fz', ULS_COMBO)
        min_shear_z = pynite_member.min_shear('Fz', ULS_COMBO)
        V_Ed = max(abs(max_shear_z), abs(min_shear_z))

        max_torsion = pynite_member.max_torque(ULS_COMBO)
        min_torsion = pynite_member.min_torque(ULS_COMBO)
        T_Ed = max(abs(max_torsion), abs(min_torsion))
        
        if spec.material.startswith('s'):
            pass
            # gamma_M = factors['gamma_M']
            # yield_strength = material_props['f_mk']
            
            # plastic_section_modulus_z = geometry.Iz / (spec.height / 2) * 1.12 if spec.height > 0 else 1e-9 # Approximation for I-sections
            
            # bending_resistance_d = plastic_section_modulus_z * yield_strength / gamma_M
            # axial_resistance_d = geometry.A * yield_strength / gamma_M
            # shear_resistance_d = (geometry.A * 0.58) * (yield_strength / np.sqrt(3)) / gamma_M
            
            # ratios['shear'] = V_Ed / shear_resistance_d
            # ratios['bending'] = M_yEd / bending_resistance_d
            # ratios['axial'] = abs(N_t0Ed) / axial_resistance_d
            # ratios['interaction'] = (abs(N_t0Ed) / axial_resistance_d) + (M_yEd / bending_resistance_d)
        else:
            if spec.shape == 'rectangular':
                k_m = 0.7
            else:
                k_m = 1

            # (EN 1995-1-1, 6.1.6)
            M_yRd, M_zRd = _calc_bending_moment_capacity(factors, material_props, spec, geometry, member_length)
            ratios['bending_y'] = (M_yEd / M_yRd * k_m) + (M_zEd / M_zRd)
            ratios['bending_z'] = (M_yEd / M_yRd) + (M_zEd / M_zRd * k_m)

            # (EN 1995-1-1, 6.1.2 - 6.1.3)
            N_t0Rd, N_t90Rd = _calc_axial_tension_capacity(factors, material_props, spec, geometry)
            ratios['axial_tension_0'] = N_t0Ed / N_t0Rd
            # ratios['axial_tension_90'] = N_t90Ed / N_t0Rd

            # (EN 1995-1-1, 6.2.3)
            bending_axial_tension_ratio_y, bending_axial_tension_ratio_z = _calc_combined_bending_axial_tension_ratio(M_yRd, M_zRd, N_t0Rd, M_yEd, M_zEd, N_t0Ed, k_m)
            ratios['bending_axial_tension_y'] = bending_axial_tension_ratio_y
            ratios['bending_axial_tension_z'] = bending_axial_tension_ratio_z

            # (EN 1995-1-1, 6.1.4 - 6.1.5)
            N_c0Rd, N_c90Rd, k_cy, k_cz, risk_buckling = _calc_axial_compression_capacity(factors, material_props, spec, geometry, member_length)
            ratios['axial_compression_0'] = N_c0Ed / N_c0Rd
            # ratios['axial_compression_90'] = N_c90Ed / (N_c90Rd * factors['config_and_deformation_factor_k_c90'])

            # (EN 1995-1-1, 6.2.4, 6.3.2)
            bending_axial_compression_ratio_y, bending_axial_compression_ratio_z = _calc_combined_bending_axial_compression_ratio(risk_buckling, M_yRd, M_zRd, N_c0Rd, M_yEd, M_zEd, N_c0Ed, k_cy, k_cz, k_m)
            ratios['bending_axial_compression_y'] = bending_axial_compression_ratio_y
            ratios['bending_axial_compression_z'] = bending_axial_compression_ratio_z

            # (EN 1995-1-1, 6.1.7)
            ratios['shear'] = _calc_shear_ratio(factors, material_props, spec, V_Ed)

            # (EN 1995-1-1, 6.1.8)
            ratios['torsion'] = _calc_torsion_ratio(factors, material_props, spec, T_Ed)

            # (EN 1995-1-1, 7.2)
            w_inst_dl = abs(pynite_member.min_deflection('dz', 'DL'))
            w_inst_ll = abs(pynite_member.min_deflection('dz', 'LL'))
            w_fin = w_inst_dl * (1 + factors['k_def']) + w_inst_ll * (1 + factors['psi_2'] * factors['k_def'])
            ratios['net_deflection'] = w_fin / (member_length / 300) # 300 is a Spanish suggestion - https://cdn.transportes.gob.es/portal-web-transportes/carreteras/normativa_tecnica/21_eurocodigos/AN_UNE-EN-1995-1-1.pdf
        
        # Bearing Check
        if member.node_i in support_node_names:
            beam_bearing_i, brick_bearing_i = _calc_compression_bearing_ratio(factors, material_props, spec, pynite_member, member_end=0)
            ratios['beam_bearing_N'] = beam_bearing_i
            ratios['brick_bearing_N'] = brick_bearing_i

        if member.node_j in support_node_names:
            beam_bearing_j, brick_bearing_j = _calc_compression_bearing_ratio(factors, material_props, spec, pynite_member, member_end=member_length)
            ratios['beam_bearing_S'] = beam_bearing_j
            ratios['brick_bearing_S'] = brick_bearing_j

        member_evaluations.append({
            'member_name': member.name,
            'material_id': member.spec.material_id,
            **ratios
            })
    
    return pd.DataFrame(member_evaluations)


def render(frame, deformed_scale=100, opacity=0.25, combo_name='ULS_Strength') -> None:
    def _set_wall_opacity(plotter, opacity=0.25):
        for actor in plotter.renderer.actors.values():
            if (hasattr(actor, 'mapper') and
                hasattr(actor.mapper, 'dataset') and
                actor.mapper.dataset.n_faces_strict > 0):
                actor.prop.opacity = opacity

    rndr = Renderer(frame)
    rndr.combo_name = combo_name
    rndr.annotation_size = 5
    rndr.render_loads = False
    rndr.render_nodes = False
    rndr.deformed_shape = True
    rndr.deformed_scale = deformed_scale
    rndr.post_update_callbacks.append(lambda plotter: _set_wall_opacity(plotter, opacity=opacity))
    rndr.render_model()


# def progress_callback(res):
#     pbar.update(1)


# def objective(params):
#     param_dict = {dimension.name: value for dimension, value in zip(space, params)}
#     try:
#         hyperparams = {
#             'east_joists': MemberSpec(param_dict['east_material'], quantity=param_dict['east_quantity'], padding=param_dict['east_padding']),
#             'west_joists': MemberSpec(param_dict['west_material'], quantity=param_dict['west_quantity'], padding=param_dict['west_padding']),
#             'tail_joists': MemberSpec(param_dict['tail_material'], quantity=param_dict['tail_quantity'], padding=param_dict['tail_padding']),
#             'trimmers': MemberSpec(param_dict['trimmer_material'], quantity=2),
#             'header': MemberSpec(param_dict['header_material'], quantity=1),
#             'planks': MemberSpec(param_dict['plank_material']),
#         }

#         frame, _, members = create_model(hyperparams, walls=True)
#         part_evaluations = evaluate_stresses(frame, members)
#         total_cost, cuts = calculate_purchase_quantity(frame, members)

#         # Group ratios into groups
#         split_names = part_evaluations['member_name'].str.split('_')
#         part_evaluations['base_name'] = split_names.str[0]
#         part_evaluations['part_num'] = pd.to_numeric(split_names.str[1], errors='coerce').astype('Int8')

#         for base_name in part_evaluations[part_evaluations['part_num'].notna()]['base_name'].unique():
#             mask = part_evaluations['base_name'] == base_name
#             parts = part_evaluations.loc[mask].sort_values('part_num')
#             part_nums = parts['part_num'].values
#             gaps = np.where(np.diff(part_nums) > 1)[0]

#             group_id = 0
#             current_group = f"{base_name}_{chr(ord('A') + group_id)}"
#             for idx, row_idx in enumerate(parts.index):
#                 part_evaluations.at[row_idx, 'member_group'] = current_group
#                 if idx in gaps:
#                     group_id += 1
#                     current_group = f"{base_name}_{chr(ord('A') + group_id)}"

#         part_evaluations['member_group'] = part_evaluations['member_group'].fillna(part_evaluations['base_name'])
#         num_cols = part_evaluations.select_dtypes(include=['number']).columns.tolist()
#         member_evaluations = part_evaluations[num_cols + ['member_group']].groupby('member_group').mean()
#         member_evaluations = member_evaluations.drop('part_num', axis=1)
        
#         max_ratio = member_evaluations.max().max()
#         mean_ratio = member_evaluations.mean().mean()
#         score = total_cost * mean_ratio * max_ratio

#     except Exception as e:
#         warnings.warn(f"An exception occurred with params {param_dict}: {e}")
#         score = 1e12
#         total_cost = float('inf')
#         member_evaluations = pd.DataFrame()
#         cuts = {}
#         max_ratio = float('inf')

#     run_result = {
#         **param_dict,
#         'total_cost': total_cost,
#         'max_ratio': max_ratio,
#         'mean_ratio': mean_ratio,
#         'score': score,
#         'cuts': cuts,
#         'score': score,
#         'member_evaluations': member_evaluations,
#     }
#     evaluations.append(run_result)

#     return score

In [ ]:
    # Units are mm, N, and MPa (N/mm²)
    INPUT_PARAMS, MATERIAL_STRENGTHS, MATERIAL_CATALOG, CONNECTORS, EUROCODE_FACTORS = prep_data()

    DL_COMBO = 'DL'
    LL_COMBO = 'LL'
    ULS_COMBO = 'ULS_Strength'

    viable_beams = MATERIAL_CATALOG[MATERIAL_CATALOG['viable_connector']]
    beam_ids = viable_beams[viable_beams['type'] == 'beam']['id'].unique().tolist()
    double_ids = viable_beams[viable_beams['type'] == 'double']['id'].unique().tolist()
    floor_ids = MATERIAL_CATALOG[MATERIAL_CATALOG['type'] == 'floor']['id'].unique().tolist()

    max_west_padding = math.ceil(INPUT_PARAMS.opening_x_start / 2)
    max_east_padding = math.ceil((INPUT_PARAMS.room_length - (INPUT_PARAMS.opening_x_start + INPUT_PARAMS.opening_length)) / 2)
    max_tail_padding = math.ceil(INPUT_PARAMS.opening_length / 3)

    beam_max_base = MATERIAL_CATALOG[MATERIAL_CATALOG['id'].isin(beam_ids)].base.max()
    max_west_quantity = math.floor(max_west_padding / beam_max_base)
    max_east_quantity = math.floor(max_east_padding / beam_max_base)
    max_tail_quantity = math.floor(max_tail_padding / beam_max_base)

    space = [
        Categorical(beam_ids, name='east_material'),
        Integer(1, max_east_quantity, name='east_quantity'),
        Integer(0, max_east_padding, name='east_padding'),

        Categorical(beam_ids, name='west_material'),
        Integer(1, max_west_quantity, name='west_quantity'),
        Integer(0, max_west_padding, name='west_padding'),

        Categorical(beam_ids, name='tail_material'),
        Integer(1, max_tail_quantity, name='tail_quantity'),
        Integer(0, max_tail_padding, name='tail_padding'),
        
        Categorical(beam_ids + double_ids, name='trimmer_material'),
        Categorical(beam_ids + double_ids, name='header_material'),
        Categorical(floor_ids, name='plank_material'),
    ]

    if 'pbar' in globals():
        pbar.close()

    n_initial_points = 2**10
    n_calls = round(n_initial_points * 1.5)
    # n_calls = n_initial_points
    pbar = tqdm(total=n_calls)

    evaluations = []
    result = gp_minimize(
        func=objective,
        dimensions=space,
        n_calls=n_calls,
        random_state=1,
        initial_point_generator='sobol',
        n_initial_points=n_initial_points,
        callback=[progress_callback]
    )

In [ ]:
import nevergrad as ng
import pandas as pd
import numpy as np
import warnings
import math
from tqdm import tqdm
import multiprocessing as mp

_GLOBAL_EVALUATIONS = []

def objective_function(east_material, east_quantity, east_padding,
                      west_material, west_quantity, west_padding,
                      tail_material, tail_quantity, tail_padding,
                      trimmer_material, header_material, plank_material):
    
    param_dict = {
        'east_material': east_material,
        'east_quantity': east_quantity,
        'east_padding': east_padding,
        'west_material': west_material,
        'west_quantity': west_quantity,
        'west_padding': west_padding,
        'tail_material': tail_material,
        'tail_quantity': tail_quantity,
        'tail_padding': tail_padding,
        'trimmer_material': trimmer_material,
        'header_material': header_material,
        'plank_material': plank_material,
    }
    
    try:
        hyperparams = {
            'east_joists': MemberSpec(east_material, quantity=east_quantity, padding=east_padding),
            'west_joists': MemberSpec(west_material, quantity=west_quantity, padding=west_padding),
            'tail_joists': MemberSpec(tail_material, quantity=tail_quantity, padding=tail_padding),
            'trimmers': MemberSpec(trimmer_material, quantity=2),
            'header': MemberSpec(header_material, quantity=1),
            'planks': MemberSpec(plank_material),
        }

        frame, _, members = create_model(hyperparams, walls=True)
        part_evaluations = evaluate_stresses(frame, members)
        total_cost, cuts = calculate_purchase_quantity(frame, members)

        # Group ratios into groups
        split_names = part_evaluations['member_name'].str.split('_')
        part_evaluations['base_name'] = split_names.str[0]
        part_evaluations['part_num'] = pd.to_numeric(split_names.str[1], errors='coerce').astype('Int8')

        for base_name in part_evaluations[part_evaluations['part_num'].notna()]['base_name'].unique():
            mask = part_evaluations['base_name'] == base_name
            parts = part_evaluations.loc[mask].sort_values('part_num')
            part_nums = parts['part_num'].values
            gaps = np.where(np.diff(part_nums) > 1)[0]

            group_id = 0
            current_group = f"{base_name}_{chr(ord('A') + group_id)}"
            for idx, row_idx in enumerate(parts.index):
                part_evaluations.at[row_idx, 'member_group'] = current_group
                if idx in gaps:
                    group_id += 1
                    current_group = f"{base_name}_{chr(ord('A') + group_id)}"

        part_evaluations['member_group'] = part_evaluations['member_group'].fillna(part_evaluations['base_name'])
        num_cols = part_evaluations.select_dtypes(include=['number']).columns.tolist()
        member_evaluations = part_evaluations[num_cols + ['member_group']].groupby('member_group').mean()
        member_evaluations = member_evaluations.drop('part_num', axis=1)
        
        max_ratio = member_evaluations.max().max()
        mean_ratio = member_evaluations.mean().mean()
        score = total_cost * mean_ratio * max_ratio

    except Exception as e:
        warnings.warn(f"An exception occurred with params {param_dict}: {e}")
        score = 1e12
        total_cost = float('inf')
        member_evaluations = pd.DataFrame()
        cuts = {}
        max_ratio = float('inf')
        mean_ratio = float('inf')

    run_result = {
        **param_dict,
        'total_cost': total_cost,
        'max_ratio': max_ratio,
        'mean_ratio': mean_ratio,
        'score': score,
        'cuts': cuts,
        'member_evaluations': member_evaluations,
    }
    _GLOBAL_EVALUATIONS.append(run_result)

    return score


def optimize_with_nevergrad(space_config, n_calls=2048, 
                            n_workers=1, algorithm='NGOpt'):
    """
    Run optimization using Nevergrad
    
    Parameters:
    -----------
    space_config : dict
        Dictionary with parameter specifications
    n_calls : int
        Total number of evaluations
    n_workers : int
        Number of parallel workers (1 = sequential)
    algorithm : str
        Nevergrad optimizer: 'NGOpt', 'CMA', 'TwoPointsDE', 'PSO', 'OnePlusOne'
    
    Returns:
    --------
    dict with 'best_params', 'best_score', 'optimizer'
    """
    
    # Build parametrization
    param_dict = {}
    for param_name, param_spec in space_config.items():
        if param_spec['type'] == 'categorical':
            param_dict[param_name] = ng.p.Choice(param_spec['choices'])
        elif param_spec['type'] == 'integer':
            param_dict[param_name] = ng.p.Scalar(
                lower=param_spec['lower'],
                upper=param_spec['upper']
            ).set_integer_casting()
    
    parametrization = ng.p.Instrumentation(**param_dict)
    
    # Select optimizer
    optimizer_class = getattr(ng.optimizers, algorithm)
    optimizer = optimizer_class(parametrization=parametrization, budget=n_calls, num_workers=n_workers)
    
    # Run optimization
    print(f"Running {algorithm} optimization with {n_calls} evaluations and {n_workers} workers...")
    
    # Sequential optimization (parallel is complex with multiprocessing)
    for _ in tqdm(range(n_calls)):
        x = optimizer.ask()
        value = objective_function(**x.kwargs)
        optimizer.tell(x, value)
    
    recommendation = optimizer.provide_recommendation()
    
    return {
        'best_params': recommendation.kwargs,
        'best_score': optimizer.current_bests["minimum"].mean,
        'optimizer': optimizer
    }


def optimize_with_cma_es(space_config, n_calls=2048):
    """
    Alternative: Pure CMA-ES implementation (requires pycma package)
    Better for continuous-like problems, handles integers via rounding
    
    Note: Install with: pip install cma
    """
    import cma
    
    # Extract bounds and create encoding/decoding
    param_names = []
    bounds_lower = []
    bounds_upper = []
    categorical_indices = {}
    
    for i, (param_name, param_spec) in enumerate(space_config.items()):
        param_names.append(param_name)
        
        if param_spec['type'] == 'categorical':
            # Encode categorical as integer index
            categorical_indices[i] = param_spec['choices']
            bounds_lower.append(0)
            bounds_upper.append(len(param_spec['choices']) - 1)
        else:
            bounds_lower.append(param_spec['lower'])
            bounds_upper.append(param_spec['upper'])
    
    def decode_and_evaluate(x):
        """Convert CMA-ES continuous values to actual parameters"""
        params = {}
        for i, (param_name, value) in enumerate(zip(param_names, x)):
            if i in categorical_indices:
                # Round to nearest integer and map to categorical
                idx = int(round(np.clip(value, bounds_lower[i], bounds_upper[i])))
                params[param_name] = categorical_indices[i][idx]
            else:
                # Round to integer
                params[param_name] = int(round(value))
        
        return objective_function(**params)
    
    # Initial guess (middle of bounds)
    x0 = [(l + u) / 2 for l, u in zip(bounds_lower, bounds_upper)]
    
    # Standard deviation (scale of initial exploration)
    sigma0 = 0.3
    
    # CMA-ES options
    opts = {
        'bounds': [bounds_lower, bounds_upper],
        'maxfevals': n_calls,
        'verb_disp': 1,
        'verb_log': 0,
        'popsize': 32,  # Population size, adjust based on dimensionality
    }
    
    print(f"Running CMA-ES optimization with {n_calls} evaluations...")
    
    es = cma.CMAEvolutionStrategy(x0, sigma0, opts)
    
    with tqdm(total=n_calls) as pbar:
        while not es.stop():
            solutions = es.ask()
            fitness_values = [decode_and_evaluate(x) for x in solutions]
            es.tell(solutions, fitness_values)
            pbar.update(len(solutions))
    
    best_x = es.result.xbest
    
    # Decode best solution
    best_params = {}
    for i, (param_name, value) in enumerate(zip(param_names, best_x)):
        if i in categorical_indices:
            idx = int(round(np.clip(value, bounds_lower[i], bounds_upper[i])))
            best_params[param_name] = categorical_indices[i][idx]
        else:
            best_params[param_name] = int(round(value))
    
    return {
        'best_params': best_params,
        'best_score': es.result.fbest,
        'cma_result': es.result
    }


# ============================================================================
# MAIN OPTIMIZATION SCRIPT
# ============================================================================
if __name__ == '__main__':
    # Units are mm, N, and MPa (N/mm²)
    INPUT_PARAMS, MATERIAL_STRENGTHS, MATERIAL_CATALOG, CONNECTORS, EUROCODE_FACTORS = prep_data()

    DL_COMBO = 'DL'
    LL_COMBO = 'LL'
    ULS_COMBO = 'ULS_Strength'

    viable_beams = MATERIAL_CATALOG[MATERIAL_CATALOG['viable_connector']]
    beam_ids = viable_beams[viable_beams['type'] == 'beam']['id'].unique().tolist()
    double_ids = viable_beams[viable_beams['type'] == 'double']['id'].unique().tolist()
    floor_ids = MATERIAL_CATALOG[MATERIAL_CATALOG['type'] == 'floor']['id'].unique().tolist()

    max_west_padding = math.ceil(INPUT_PARAMS.opening_x_start / 2)
    max_east_padding = math.ceil((INPUT_PARAMS.room_length - (INPUT_PARAMS.opening_x_start + INPUT_PARAMS.opening_length)) / 2)
    max_tail_padding = math.ceil(INPUT_PARAMS.opening_length / 3)

    beam_max_base = MATERIAL_CATALOG[MATERIAL_CATALOG['id'].isin(beam_ids)].base.max()
    max_west_quantity = math.floor(max_west_padding / beam_max_base)
    max_east_quantity = math.floor(max_east_padding / beam_max_base)
    max_tail_quantity = math.floor(max_tail_padding / beam_max_base)

    # Define search space
    space_config = {
        'east_material': {'type': 'categorical', 'choices': beam_ids},
        'east_quantity': {'type': 'integer', 'lower': 1, 'upper': max_east_quantity},
        'east_padding': {'type': 'integer', 'lower': 0, 'upper': max_east_padding},
        
        'west_material': {'type': 'categorical', 'choices': beam_ids},
        'west_quantity': {'type': 'integer', 'lower': 1, 'upper': max_west_quantity},
        'west_padding': {'type': 'integer', 'lower': 0, 'upper': max_west_padding},
        
        'tail_material': {'type': 'categorical', 'choices': beam_ids},
        'tail_quantity': {'type': 'integer', 'lower': 1, 'upper': max_tail_quantity},
        'tail_padding': {'type': 'integer', 'lower': 0, 'upper': max_tail_padding},
        
        'trimmer_material': {'type': 'categorical', 'choices': beam_ids + double_ids},
        'header_material': {'type': 'categorical', 'choices': beam_ids + double_ids},
        'plank_material': {'type': 'categorical', 'choices': floor_ids},
    }

    n_calls = 2048
    
    # Clear global evaluations list
    _GLOBAL_EVALUATIONS.clear()
    
    # Option 1: Nevergrad with automatic algorithm selection (RECOMMENDED)
    result = optimize_with_nevergrad(
        space_config, 
        n_calls=n_calls,
        n_workers=-1,
        algorithm='CMA'  # Try also: 'CMA', 'NGOpt', 'TwoPointsDE', 'PSO', 'DE'
    )
    
    # Option 2: Pure CMA-ES (uncomment to use instead)
    # result = optimize_with_cma_es(space_config, n_calls=n_calls)
    
    print("\n" + "="*80)
    print("OPTIMIZATION COMPLETE")
    print("="*80)
    print(f"\nBest Score: {result['best_score']:.2f}")
    print("\nBest Parameters:")
    for param, value in result['best_params'].items():
        print(f"  {param}: {value}")
    
    # Save results from global evaluations list
    results_df = pd.DataFrame(_GLOBAL_EVALUATIONS)

In [ ]:
results_df

In [ ]:
# Single run
INPUT_PARAMS, MATERIAL_STRENGTHS, MATERIAL_CATALOG, CONNECTORS, EUROCODE_FACTORS = prep_data()

hyperparams = {
    'east_joists' : MemberSpec('c24_45x75', quantity=2, padding=100),
    'west_joists' : MemberSpec('c24_45x75', quantity=2, padding=100),
    'tail_joists' : MemberSpec('c24_45x75', quantity=2, padding=0),
    'trimmers' : MemberSpec('c24_45x75', quantity=2),
    'header' : MemberSpec('c24_45x75', quantity=1),
    'planks' : MemberSpec('c18_200x21'),
    }

DL_COMBO = 'DL'
LL_COMBO = 'LL'
ULS_COMBO = 'ULS_Strength'

frame, nodes, members = create_model(hyperparams, walls=True)
part_evaluations = evaluate_stresses(frame, members)
total_cost, cuts = calculate_purchase_quantity(frame, members)

In [ ]:
pd.DataFrame(evaluations)#.iloc[0]['member_evaluations']

In [ ]:
def create_member_groups(df):
    split_names = df['member_name'].str.extract(r'^([a-zA-Z]+)(?:_(\d+))?$')
    df['base_name'] = split_names[0]
    df['part_num'] = split_names[1].astype(float)  # NaN for no underscore
    
    df['member_group'] = df['base_name']
    for base_name in df[df['part_num'].notna()]['base_name'].unique():
        mask = df['base_name'] == base_name
        parts = df.loc[mask].sort_values('part_num')
        
        part_nums = parts['part_num'].values
        gaps = np.where(np.diff(part_nums) > 1)[0]
        
        group_id = 0
        current_group = f"{base_name}_g{group_id}"
        
        for idx, row_idx in enumerate(parts.index):
            df.at[row_idx, 'member_group'] = current_group
            
            if idx in gaps:
                group_id += 1
                current_group = f"{base_name}_g{group_id}"
    
    return df

# Use it:
all_evals = create_member_groups(member_evaluations)
num_cols = all_evals.select_dtypes(include=['number']).columns.tolist()
member_evals = all_evals[num_cols + ['member_group']].groupby('member_group').mean()
member_evals

In [ ]:
member_evaluations

In [ ]:
df = pd.DataFrame(evaluations).sort_values(by='score')
df.iloc[0]['member_evaluations']